In [1]:
from SRCNN_different_specs import RunSRCNN, SRCNN
from custom_image_dataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torch import nn

In [2]:
DATASET_ROOT="./datasets/"
DATASET_NAME = "Flickr2K"

In [3]:
%%time
transform_x = transforms.Compose([
    transforms.Resize((11, 11), interpolation=Image.BICUBIC),
    transforms.Resize((33, 33), interpolation=Image.BICUBIC)
])

transform_y = transforms.Compose([
    transforms.CenterCrop((33, 33))
])

# Set up the data loaders
train_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_train", transform=transform_x, target_transform=transform_y)
validation_data_set = CustomImageDataset(
    img_dir=f"{DATASET_ROOT}/{DATASET_NAME}_validation", transform=transform_x, target_transform=transform_y)

# Report split sizes
print('Training set has {} instances'.format(len(train_data_set)))
print('Validation set has {} instances'.format(len(validation_data_set)))

100%|██████████| 2650/2650 [00:36<00:00, 73.41it/s]


Training set has 162186 instances
Validation set has 53000 instances
CPU times: total: 19.1 s
Wall time: 3min 38s


In [4]:
%%time

# https://towardsdatascience.com/7-tips-for-squeezing-maximum-performance-from-pytorch-ca4a40951259
pin_memory = False
batch_size = round(0.35*(2**13))

train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, pin_memory=pin_memory) # TODO: aumentar batch_size p/ 2^12
validation_dataloader = DataLoader(validation_data_set, batch_size=batch_size, pin_memory=pin_memory)

CPU times: total: 19.7 s
Wall time: 25.6 s


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-3

model_f2_5 = SRCNN(f2=5).to(device)
optimizer_f2_5 = torch.optim.Adam(model_f2_5.parameters(), lr=lr)
scheduler_f2_5 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_5,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=60)

model_f2_1 = SRCNN(f2=1).to(device)
optimizer_f2_1 = torch.optim.Adam(model_f2_1.parameters(), lr=lr)
scheduler_f2_1 = torch.optim.lr_scheduler.LinearLR(
    optimizer_f2_1,
    start_factor=1.0,
    end_factor=0.01,
    total_iters=60)


In [6]:
run_srcnn = RunSRCNN()

In [7]:
%%time
run_srcnn.train(
    model=model_f2_1,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer_f2_1,
    epochs=100,
    scheduler=scheduler_f2_1,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


  0%|          | 0/57 [00:00<?, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 57/57 [03:55<00:00,  4.13s/it]


Training Error: 
 Avg loss: 0.041828 



100%|██████████| 19/19 [00:46<00:00,  2.45s/it]


Validation Error: 
 Avg loss: 0.016614 

Learning rate (antes): 0.001
Learning rate (depois): 0.0009835

epoch 1
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


...

Validation Error: 
 Avg loss: 0.003622 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05

epoch 99
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


100%|██████████| 57/57 [02:57<00:00,  3.11s/it]


Training Error: 
 Avg loss: 0.007387 



100%|██████████| 19/19 [00:38<00:00,  2.04s/it]

Validation Error: 
 Avg loss: 0.003622 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05
CPU times: total: 5h 34min 18s
Wall time: 7h 15min


In [8]:
df_f2_1 = run_srcnn.get_model_df()
df_f2_1.tail()

,epoch,time,lr,train_loss,validation_loss
95,95,258.085769,0.00001,0.007389,0.003624
96,96,261.805513,0.00001,0.007389,0.003623
97,97,256.213686,0.00001,0.007388,0.003622
98,98,258.176599,0.00001,0.007389,0.003622
99,99,279.710755,0.00001,0.007387,0.003622


In [9]:
run_srcnn.save_model_df('./results/srcnn/dataframes/model_f2_1.csv')
run_srcnn.save_model_weights(model_f2_1, './results/srcnn/dataframes/model_f2_1.pth')

In [10]:
%%time

run_srcnn.clear_all()

run_srcnn.train(
    model=model_f2_5,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    optimizer=optimizer_f2_5,
    epochs=100,
    scheduler=scheduler_f2_5,
    loss_fn=nn.MSELoss(),
)


epoch 0
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


  0%|          | 0/57 [00:00<?, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 57/57 [03:48<00:00,  4.02s/it]


Training Error: 
 Avg loss: 0.028788 



100%|██████████| 19/19 [00:40<00:00,  2.14s/it]


Validation Error: 
 Avg loss: 0.005754 

Learning rate (antes): 0.001
Learning rate (depois): 0.0009835

epoch 1
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


...

Validation Error: 
 Avg loss: 0.003254 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05

epoch 99
-------------------------------
Training on 162186 samples...
Batch size: 2867
Number of batches: 57


100%|██████████| 57/57 [03:46<00:00,  3.97s/it]


Training Error: 
 Avg loss: 0.006895 



100%|██████████| 19/19 [00:43<00:00,  2.30s/it]

Validation Error: 
 Avg loss: 0.003254 

Learning rate (antes): 1.0000000000000028e-05
Learning rate (depois): 1.0000000000000028e-05
CPU times: total: 6h 54min 50s
Wall time: 9h 17min 27s


In [11]:
df_f2_5 = run_srcnn.get_model_df()
df_f2_5.tail()

,epoch,time,lr,train_loss,validation_loss
95,95,323.963578,0.00001,0.006893,0.003254
96,96,351.199800,0.00001,0.006893,0.003255
97,97,319.389475,0.00001,0.006892,0.003255
98,98,325.845934,0.00001,0.006891,0.003254
99,99,325.318786,0.00001,0.006895,0.003254


In [12]:
run_srcnn.save_model_df('./results/srcnn/dataframes/model_f2_5.csv')
run_srcnn.save_model_weights(model_f2_1, './results/srcnn/dataframes/model_f2_5.pth')

In [13]:
path = f"{DATASET_ROOT}/Set14"
run_srcnn.compare_models([{"name": "model_f2_1", "model": model_f2_1}, {"name": "model_f2_5", "model": model_f2_5}], path)

0it [00:00, ?it/s]C:\Users\Pedro\anaconda3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
14it [00:02,  6.76it/s]
14it [00:01,  9.28it/s]


,model_f2_1,model_f2_5
PSNR,24.557705,24.907141
Bicubic PSNR,24.496832,24.496832
SSIM,0.714441,0.729018
Bicubic SSIM,0.731670,0.731670
